In [37]:
try:
    import google.colab
    google.colab.drive.mount('/content/drive/')
except:
    pass

from sklearn import svm
import numpy as np
from mlpath import mlquest as mlq
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import f1_score
import albumentations.pytorch as AP
import pytorch_lightning as pl
import os
import cv2
from tqdm import tqdm
import albumentations as A
import torchvision.models as models
import torch
import glob

train_path = '../content/drive/MyDrive/Train'
test_path = '../content/drive/MyDrive/Test'

### Read Data & Preprocess

In [39]:
def preprocess_img(img, new_size=256, normalize=True, more_transforms=None):
    transforms_list = [
        A.SmallestMaxSize(max_size=new_size),
        A.CenterCrop(new_size, new_size),
    ]
    if normalize: transforms_list.append(A.Normalize(mean=[0.277, 0.277, 0.277], std=[0.254, 0.246, 0.223]))
    if more_transforms: transforms_list.extend(more_transforms)
    transform = A.Compose(transforms_list)
    img_p = transform(image=img)['image']
    return img_p

In [35]:
def read_data(train_path, gray=False, new_size= 256, normalize=False, transpose=False, more_transforms=None):
    flooded_path = train_path + '/flooded/*.jpg'
    non_flooded_path = train_path + '/non-flooded/*.jpg'
    print(flooded_path)

    x_data, y_data= [], []
    
    for filename in tqdm(sorted(glob.glob(flooded_path))):
            img = cv2.imread(filename, 0 if gray else cv2.COLOR_BGR2RGB)
            img_p = preprocess_img(img, new_size, normalize, more_transforms)
            if transpose: img_p = img_p.transpose(2,0,1)
            x_data.append(img_p)
            y_data.append(1)

        
    for filename in tqdm(sorted(glob.glob(non_flooded_path))):
            img = cv2.imread(filename, 0 if gray else cv2.COLOR_BGR2RGB)
            img_p = preprocess_img(img, new_size, normalize,  more_transforms)
            if transpose: img_p = img_p.transpose(2,0,1)
            x_data.append(img_p)
            y_data.append(0)

    x_data,  y_data = np.array(x_data), np.array(y_data)

    
    return x_data, y_data


def read_test(test_path, gray=False, new_size=256, normalize=False, transpose=False, more_transforms=None):
        test_path = test_path + '/*.jpg'
        
        x_data= []
        
        for filename in tqdm(sorted(glob.glob(test_path))):
                img = cv2.imread(filename, 0 if gray else cv2.COLOR_BGR2RGB)
                img_p = preprocess_img(img, new_size, normalize, more_transforms)
                if transpose: img_p = img_p.transpose(2,0,1)
                x_data.append(img_p)
        
        x_test = np.array(x_data)
        
        return x_test

### Feature Extraction

In [29]:

class ConvNet(pl.LightningModule):
    '''
    ShuffleNetV2 Model
    '''
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = models.shufflenet_v2_x1_0(pretrained=True)
        self.model.eval()
        
    def forward(self, x):
        '''
        Forward method to pass a given input batch to the model and return its predictions
        '''
        x = self.model(x.unsqueeze(0))
        return x

model = ConvNet()


def deep_features(img):
    '''
    Produces a feature vector for the image using a pretrained model
    '''
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    img = img.to(device)
    model = ConvNet().to(device)
    model.eval()
    feat = model(img)
    return feat.detach().cpu().numpy()



def apply_deep_features(x_data):
    '''
    Simply applies the deep_feature function to each image in x_data
    '''
    x_data_f = []
    for x in tqdm(x_data):
        feat = deep_features(x)
        feat = np.squeeze(feat)
        x_data_f.append(feat)
    return np.array(x_data_f)



/opt/anaconda3/envs/M1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/M1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Model Pipeline

In [30]:
more_transforms = [AP.transforms.ToTensorV2()]          # PyTorch deals with tensors on the GPU not numpy arrays
x_data, y_data = read_data(train_path, gray=False, normalize=True, new_size=224, more_transforms=more_transforms)

x_data_f = apply_deep_features(x_data)


model = svm.SVC(C=60, kernel='rbf')
model.fit(x_data_f, y_data)

../DataFiles//flooded/*.jpg


100%|██████████| 461/461 [00:33<00:00, 13.79it/s]
/var/folders/nq/f_nplzp52qx_hrrhyg44xf4c0000gn/T/ipykernel_23311/462334224.py:23: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  x_data,  y_data = np.array(x_data), np.array(y_data)
/var/folders/nq/f_nplzp52qx_hrrhyg44xf4c0000gn/T/ipykernel_23311/462334224.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_data,  

(922, 1000)


SVC(C=60)

In [33]:
model.predict(x_data_f)
f1_score(y_data, model.predict(x_data_f))

# read svm_model.pkl
import pickle
with open('../content/drive/MyDrive/svm_model.pkl', 'rb') as f:
    model = pickle.load(f)



1.0

#### Testing

In [36]:
x_test = read_test(test_path, gray=False, normalize=True, new_size=224, more_transforms=more_transforms)
x_test_f = apply_deep_features(x_test)
model.predict(x_test_f)

with open('predictions.txt', 'w') as f:
    for pred in model.predict(x_test_f):
        f.write(str(pred) + '\n')

100%|██████████| 922/922 [00:35<00:00, 26.30it/s] 
/var/folders/nq/f_nplzp52qx_hrrhyg44xf4c0000gn/T/ipykernel_23311/375446012.py:40: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  x_test = np.array(x_data)
/var/folders/nq/f_nplzp52qx_hrrhyg44xf4c0000gn/T/ipykernel_23311/375446012.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test = np.array(x_data)
  0%|    